In [1]:
from deeplotx import SoftmaxRegression, LongformerEncoder
from util import NUM_CLASSES
lf_encoder = LongformerEncoder(model_name_or_path='severinsimmler/xlm-roberta-longformer-base-16384')

[DEBUG] 2025-08-03 15:15:06,656 deeplotx.embedding : LongformerEncoder initialized on device: cuda.


In [2]:
import torch
from deeplotx.util import sha256
from vortezwohl.cache import LRUCache

CACHE = LRUCache(capacity=16384)

def encode(text: str) -> torch.Tensor:
    key = sha256(text)
    if key in CACHE:
        return CACHE[key]
    emb = lf_encoder.encode(text, cls_only=False).mean(dim=-2, dtype=model.dtype)
    CACHE[key] = emb
    return emb

In [4]:
import json
from random import shuffle

with open('./data/datasets.json', 'r', encoding='utf-8') as f:
    datasets = json.load(f)
    
train_dataset = datasets['train']
valid_dataset = datasets['valid']
test_dataset = datasets['test']
    
shuffle(train_dataset)
shuffle(valid_dataset)
shuffle(test_dataset)
print('Dataset shuffled', list(train_dataset[:23]))

Dataset shuffled [['RSI LA1', 2], ['Frosolone', 3], ['estende', 0], ['Grant', 1], ['Beaurains', 3], ['attristé', 0], ['Nankin', 3], ['Salvatore Aurelio', 1], ['Disentis', 3], ['passo del Grimsel', 3], ['Hodotermidae', 4], ['Ангарский Ермак', 2], ['come loro leader', 4], ['West Germany', 3], ['Eastwell Park', 3], ['précipitations occasionnelles', 3], ['Pirgos', 3], ['Bacchanalia', 4], ['Dallas en ,', 2], ['Manuel Pérez Barriopedro', 1], ['SIXAXIS', 4], ['Battle of Salamis', 4], ['LNA', 2]]


## 训练

In [ ]:
model = SoftmaxRegression(input_dim=768, output_dim=NUM_CLASSES, num_heads=4, num_layers=3, expansion_factor=1.25, bias=True, dropout_rate=0.2, head_layers=2)
print(model)

In [37]:
# train
from torch import nn, optim
from torch.utils.tensorboard import SummaryWriter

In [38]:
train_step = 0
valid_step = 0
writer = SummaryWriter()

acc_train_loss = 0.
acc_valid_loss = 0.
eval_interval = 2000
log_interval = 200
valid_log_interval = 50

In [ ]:
from random import randint
from util import one_hot

elastic_net_param = {
    'alpha': 2e-4,
    'rho': 0.2
}
learning_rate = 2e-6
num_epochs = 1500
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    model.train()
    for i, (_token, _label) in enumerate(train_dataset):
        _one_hot_label = one_hot(_label).to(model.dtype).to(model.device)
        outputs = model.forward(encode(_token))
        loss = loss_function(outputs, _one_hot_label) + model.elastic_net(alpha=elastic_net_param['alpha'], rho=elastic_net_param['rho'])
        acc_train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if train_step % log_interval == 0 and train_step > 0:
            writer.add_scalar('train/loss', acc_train_loss / log_interval, train_step)
            print(f'- Train Step {train_step} Loss {acc_train_loss / log_interval} \\'
                  f'\nToken={_token}'
                  f'\nPred={outputs.tolist()}'
                  f'\nLabel={_one_hot_label.tolist()}', flush=True)
            acc_train_loss = 0.
        train_step += 1
        if train_step % eval_interval == 0:
            model.eval()
            rand_idx = randint(0, len(valid_dataset) - 501)
            with torch.no_grad():
                for _i, (__token, __label) in enumerate(valid_dataset[rand_idx: rand_idx + 500]):
                    _one_hot_label = one_hot(__label).to(model.dtype).to(model.device)
                    outputs = model.forward(encode(__token))
                    loss = loss_function(outputs, _one_hot_label)
                    acc_valid_loss += loss.item()
                    if valid_step % valid_log_interval == 0 and valid_step > 0:
                        writer.add_scalar('valid/loss', acc_valid_loss / valid_log_interval, valid_step)
                        print(f'- Valid Step {valid_step} Loss {acc_valid_loss / valid_log_interval} \\'
                              f'\nToken={__token}'
                              f'\nPred={outputs.tolist()}'
                              f'\nLabel={_one_hot_label.tolist()}', flush=True)
                        acc_valid_loss = 0.
                    valid_step += 1
            model.train()

In [ ]:
model.eval()
test_tokens = ['Mike', 'John', 'Smith', 'London', 'NYC', 'HongKong', 'China', 'South Africa', 'Korea']
with torch.no_grad():
    for _tok in test_tokens:
        _dist = model.forward(encode(_tok))
        print(f'Token={_tok}, Class={torch.argmax(_dist)}', flush=True)
model.train()
'Test finished.'